### Preparing assembly files for binning
To create a depth processing file, reads must be re-aligned to the contigs. This has been done using bowtie2 (can also be done using BWA). The next step would be to create a depth file with MetaBat2, convert that to be suitable for CONCOCT and MaxBin2, and then process these into bins. 

This is all assuming you have installed all of the softwares mentioned here. Use conda for quick install. If needed, the documentation for everything can be found here:

MetaBat2: 

MaxBin2:

CONCOCT: https://github.com/BinPro/CONCOCT

CheckM: https://github.com/Ecogenomics/CheckM/wiki

#### MetaBat2
The first piece of code here generates a fairly simple text file for the coverage of these files. The next set of code runs MetaBat2  (v2.10.2) using minContig 2500, minCV 1.0, minCVSum 1.0, maxP 95%, minS 60, and maxEdges 200. It gathers all mapping information into a single depth file, so you can use your 1 file in the next analysis

In [1]:
#this creates a depth file for MetaBat
jgi_summarize_bam_contig_depths --outputDepth ../data/working/metabat_depth_co-assembly1.txt ../../03_mapping/data/results/*.bam
done

SyntaxError: invalid syntax (<ipython-input-1-5d2ab8b6c55f>, line 2)

In [11]:
#this is the actual MetaBat2 script
metabat2 -i ../../02_assembly/data/contigs-anvio/"$f".contigs-fixed.fa -a depth.txt \
-o ../data/working/metabat_depth_"$f".txt \
-v \
-m 1500 \
-s 200 \#this is much lower than they do, kinda worried about this one... should I bin all the samples at once or bin them all seprarately? \
-l --unbinned \

done

SyntaxError: invalid syntax (<ipython-input-11-87bf656f4fdd>, line 2)

 #### CONCOCT
 This set of commands runs CONCOCT in its standard mode. It first creates a depth/coverage file for itself to use and then runs CONCOCT, with the standard settings. This means k-mer value is set to 4, minimum contig length is 1000, and CONCOCT runs on the exact amount of slots given to it by Hydra. 
 
CONCOCT creates a depth file out of the coverance created in the mapping step. It is key that this is all in the correct places before proceeding with binning. It creates a single file, which is then used for the complete binning process. Do keep in mind that binning might take awhile, so be prepared to let this run overnight.

IMPORTANT: in the current version of CONCOCT, you're missing a vital file, called libmkl.so. Without this file, CONCOCT will not be able to start. You can fix this issue by installing another file through Conda: 

conda install mkl

Additionally, samtools will not work properly after a fresh CONCOCT install. The easiest way to fix this is to go to your environment where you installed CONCOCT and force an update through conda. 

In [1]:
#this creates the CONCOCT depth file

#this part cuts up the contigs into 10kb pieces for CONCOCT to use 
cut_up_fasta.py ../../02_assembly/data/results/contigs_fixed/co-assembly1.contigs-fixed.fa -c 1000 -o 0 --merge_last -b ../data/working/co-assembly1_contigs_cut.bed > ../data/working/co-assembly1_contigs_cut.fa
#this part estimates contig coverage
concoct_coverage_table.py ../data/working/co-assembly1_cut.bed ../../03_mapping/data/results/*.bam > ../data/working/coverage_table_co-assembly1.tsv

#this script needs an error output! second part will fail and give no notice, causing CONCOCT to fail


SyntaxError: invalid syntax (<ipython-input-1-f36314307c24>, line 4)

In [9]:
#CONCOCT script
#make correct directories (can be omitted I think)
mkdir ../data/results/concoct_bins
mkdir ..data/working/concoct_temp

#this next bit actually runs CONCOCT itself
concoct --composition_file ../data/working/co_assembly1_cut.fa --coverage_file ../data/working/coverage_table_"$f".tsv -t $NSLOTS -b ../data/working/concoct_temp/"$f"_concoct_temp/
merge_cutup_clustering.py ../data/working/concoct_temp/"$f"_concoct_temp/clustering_gt1000.csv > ..data/working/concoct_temp/"$f"_concoct_temp/"$f"_clustering_merged.csv
mkdir ../data/results/concoct_bins/"$f"_concoct_bins
extract_fasta_bins.py "$f".contigs.fa ..data/working/concoct_temp/"$f"_concoct_temp/"$f"_clustering_merged.csv --output_path ../data/results/concoct_bins/"$f"_concoct_bins
done

SyntaxError: invalid syntax (<ipython-input-9-1ee438ed8a32>, line 3)

### Binsanity
This is another binning software that can be used. *Under construction*

### DAS_tool
This is a tool to recombine all your bins from several different algorithms into a single one, without redundancy *under construction*

### Continuing
You should now have 3 sets of bins, each created with a slightly different algorithm. It is now important to run the CheckM software with the script below and generate output files for all of them. This will inform you towards the quality of your bins and your contamination/completion rate. After this, you can proceed to the "Refine Bins" part of the workflow.

CheckM runs a check against a database to determine the levels of completeness versus contamination. These statistics are vital in determining how you want to proceed in the refinement process. 